In [2]:
# GPU availability
import tensorflow as tf
print('tf version {}'.format(tf.__version__))
print('Is GPU available {}'.format(tf.test.is_gpu_available()))
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))

tf version 1.13.1
Is GPU available False


In [9]:
# gym specifications
import gym
env_name = 'CartPole-v0'
env_name = 'Breakout-v0'
env_name = 'Pong-v0'
env = gym.make(env_name)
print(env_name)
print('observation dimensionality {}'.format(env.observation_space.shape ))
print('number of actions {}'.format(env.action_space.n))
print('meaning of actions {}'.format(env.unwrapped.get_action_meanings()))

Pong-v0
observation dimensionality (210, 160, 3)
number of actions 6
meaning of actions ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [ ]:
# Train a model
%load_ext autoreload
%autoreload 2
import gym
import os
from model import Model
from cfg import get_cfg
from utils import get_experiment_name
from utils import dotdict
import tensorflow as tf
tf.reset_default_graph()

# where to save experimental results
experiments_folder = os.path.join(os.getcwd(), 'results')
experiment_name, experiment_folder = get_experiment_name(experiments_folder)

# specify environment
env_name = 'four_rooms_maze'
env_name = 'BreakoutNoFrameskip-v4'
env_name = 'Pong-v0'
env_name = 'CartPole-v0'

# specify agent
agent_name = 'vpg'
agent_name = 'ppo'
agent_name = 'dqn'

# load default config parameters
cfg_env, cfg_agent = get_cfg(experiment_folder, env_name, agent_name)
cfg = cfg_env
cfg.update(cfg_agent)
cfg = dotdict(cfg)

# modify default config parameters
cfg.n_episodes = 1000
cfg.batch_size = 1
cfg.log_step = 1000
cfg.verbose = False

# train model
model = Model(cfg)
model.train_model()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
ep 55 st 1000 dqn e.ret.tr 18.20 ag.ls 0.76 eps 0.00 
ep 91 st 2000 dqn e.ret.tr 24.65 ag.ls 1.06 eps 0.00 
ep 112 st 3000 dqn e.ret.tr 48.25 ag.ls 3.06 eps 0.00 
ep 137 st 4000 dqn e.ret.tr 39.10 ag.ls 4.65 eps 0.00 
ep 147 st 5000 dqn e.ret.tr 75.65 ag.ls 6.34 eps 0.00 


In [11]:
# Load pretrained model
import os
from model import Model
import tensorflow as tf
tf.reset_default_graph()
experiment_folder = os.path.join(os.getcwd(), 'results/11')
model = Model.load(experiment_folder)

INFO:tensorflow:Restoring parameters from /Users/raoul/code/drl/results/11/ppo_ckpt-250
INFO:tensorflow:Restoring parameters from /Users/raoul/code/drl/results/11/baseline_ckpt-250


In [15]:
# Simulate one episode in OpenAI gym
import time 
obs, ext_return, done = model.env.reset(), 0., False
n_steps, _return = 0, 0
while not done:
    obs = model.encode_obs(obs)
    action = model.action(obs)
    obs, r, done, _ = model.env.step(action)
    _return += r
    n_steps += 1
    model.env.render()
    time.sleep(0.1)
model.env.close()
print(_return, n_steps)

84.0 85


In [20]:
# Simulate one episode in the Maze environment
import matplotlib 
%matplotlib nbagg
ext_return, n_steps, obs_list = model.simulate()
print(ext_return, n_steps, len(model.env.visited))
model.env.animate(obs_list)

AttributeError: 'AtariGame' object has no attribute 'visited'